In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
from libs.datasets import MNIST
import os
import pandas as pd
from libs.utils import montage
from libs import gif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn import preprocessing
from six.moves import cPickle as pickle
import seaborn as sns
%matplotlib inline

In [3]:
def do_pickle(dataset, set_filename):
    try:
        with open(set_filename, 'wb') as f:
            pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', set_filename, ':', e)

        
def do_unpickle(pickle_file):
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
    return data

In [4]:
data = do_unpickle("processed_data_binarized_labels.pkl")
[X_train, y_train] = data['train']
[X_test, y_test] = data['test']
[X_validation, y_validation] = data['validation']
scaler = data['scaler']
label_encoder = data['label_encoder']
y = data['y']
y_ = data['y_']

In [23]:
from copy import deepcopy
image_size = 28
num_channels = 1 # gray scale
num_labels = 10

# reshape with channel = 1
X_train = X_train.reshape(-1, image_size, image_size, num_channels)
X_test = X_test.reshape(-1, image_size, image_size, num_channels)
X_validation = X_validation.reshape(-1, image_size, image_size, num_channels)

print np.shape(X_train)
print "number of samples:", len(y_), y_[0]

(4000, 28, 28, 1)
number of samples: 5000 [0 0 0 0 1 0 0 0 0 0]


In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

## Convnet: 2 conv + 1 fully connected layer

In [30]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(X_validation)
    tf_test_dataset = tf.constant(X_test)
  
    # Variables.
    # format is [patch_size, patch_size, input_channels, output_channels
    # Conv layer: 5x5, 1 => 16
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    # Conv layer: 5x5, 16 => 16
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # Fully connected layer: 28/4*28/4*16, 1024
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    # Relu
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [27]:
num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :, :, :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), y_validation))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
Minibatch loss at step 0: 2.887535
Minibatch accuracy: 12.5%
Validation accuracy: 9.8%
Minibatch loss at step 500: 0.626386
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.120724
Minibatch accuracy: 100.0%
Validation accuracy: 90.8%
Minibatch loss at step 1500: 0.209437
Minibatch accuracy: 93.8%
Validation accuracy: 91.8%
Minibatch loss at step 2000: 0.024510
Minibatch accuracy: 100.0%
Validation accuracy: 95.0%
Minibatch loss at step 2500: 0.006584
Minibatch accuracy: 100.0%
Validation accuracy: 96.2%
Minibatch loss at step 3000: 0.007180
Minibatch accuracy: 100.0%
Validation accuracy: 96.6%
Minibatch loss at step 3500: 0.015953
Minibatch accuracy: 100.0%
Validation accuracy: 96.0%
Minibatch loss at step 4000: 0.005376
Minibatch accuracy: 100.0%
Validation accuracy: 96.6%
Minibatch loss at step 4500: 0.001707
Minibatch accuracy: 100.0%
Validation accuracy: 96.8%
Test accuracy: 90.4%
